### Inference Pipeline Testing with train_soundscapes (Random Forest)
This notebook demonstrates the pipeline testing process for the trained Random Forest model using the provided train_soundscapes dataset instead of the hidden test set.

***Objective:***
Before submitting the final inference notebook to the BirdCLEF 2025 competition, this notebook verifies whether:

- The preprocessing steps (chunking, mel feature extraction) align with training.
- The model produces valid predictions in the required format (row_id + probability vector).
- The label mapping and chunk timing convention are correctly followed.

***Data Used:***
- Input audio: train_soundscapes/
- Trained model: random_forest_model.pkl
- Label encoder: label_encoder.pkl

***Outcome:***
If this inference notebook runs successfully and produces a correctly formatted CSV file, it confirms that the pipeline is ready for Kaggle test-time inference using test_soundscapes/.

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import joblib
from tqdm import tqdm

# === CONFIG ===
TEST_DIR = "data/train_soundscapes"   # change to test_soundscapes on final run
MODEL_PATH = "random_forest_model_v2.pkl"
ENCODER_PATH = "label_encoder_v2.pkl"
SUBMISSION_PATH = "submission.csv"

CHUNK_DURATION = 5  # seconds
SR = 32000
N_MELS = 64
TRAIN_CSV_PATH = "data/train.csv"

# === Load model and label encoder ===
model = joblib.load(MODEL_PATH)
le = joblib.load(ENCODER_PATH)
species_ids = sorted(le.classes_.tolist())

# === Helper functions ===
def split_into_chunks(y, sr, chunk_duration=5):
    step = chunk_duration * sr
    return [y[i:i+step] for i in range(0, len(y), step) if len(y[i:i+step]) == step]

def waveform_to_feature(chunk, sr=SR, n_mels=N_MELS):
    mel = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mean = mel_db.mean(axis=1)
    std = mel_db.std(axis=1)
    return np.concatenate([mean, std])  # (128,)

# === Inference ===
results = []

for fname in tqdm(sorted(os.listdir(TEST_DIR))):
    if not fname.endswith(".ogg"): continue

    filepath = os.path.join(TEST_DIR, fname)
    y, _ = librosa.load(filepath, sr=SR)

    # Skip human voice trimming
    chunks = split_into_chunks(y, SR, CHUNK_DURATION)

    for i, chunk in enumerate(chunks):
        feature = waveform_to_feature(chunk)  # (128,)
        feature = feature.reshape(1, -1)
        probs = model.predict_proba(feature)[0]

        pred_vector = [0.0] * len(species_ids)
        for label_idx, prob in zip(model.classes_, probs):
            species = le.inverse_transform([label_idx])[0]
            index = species_ids.index(species)
            pred_vector[index] = prob

        end_time = (i + 1) * CHUNK_DURATION
        row_id = f"{fname[:-4]}_{end_time}"
        row = [row_id] + pred_vector
        results.append(row)

# === Save submission ===
submission_df = pd.DataFrame(results, columns=["row_id"] + species_ids)
submission_df.to_csv(SUBMISSION_PATH, index=False)
print(f"Saved predictions to {SUBMISSION_PATH}")

100%|██████████| 700/700 [04:37<00:00,  2.52it/s]


✅ Saved predictions to submission.csv
